In [1]:
%pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [2]:
!gdown "1n6IHWly66qSJfxrDSaIBcqm05jo-v6mt"
!gdown "1zToBjksCyT4DYAZ-0Oqj52nEXk0aqtdm"
!gdown "1wKZjFyWcfsTveGpuRwXHqmfFW8Zu8cIe"
!gdown "10S8tYhh1jd5VyQD7Rbw2QVBXIfw3hd1H"
!gdown "1Fs0PTy_xPsoX5bg4QTCrX6JAaQCHRzis"
!gdown "1JLa-ELhUskQINi0syf3YJWBX8jcCXdn4"

Downloading...
From: https://drive.google.com/uc?id=1n6IHWly66qSJfxrDSaIBcqm05jo-v6mt
To: /kaggle/working/rs_train_embeddings.pkl
100%|██████████████████████████████████████| 49.5M/49.5M [00:05<00:00, 9.50MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zToBjksCyT4DYAZ-0Oqj52nEXk0aqtdm
To: /kaggle/working/rs_test_embeddings.pkl
100%|██████████████████████████████████████| 12.4M/12.4M [00:01<00:00, 10.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wKZjFyWcfsTveGpuRwXHqmfFW8Zu8cIe
To: /kaggle/working/ls_train_embeddings.pkl
100%|██████████████████████████████████████| 49.5M/49.5M [00:05<00:00, 9.28MB/s]
Downloading...
From: https://drive.google.com/uc?id=10S8tYhh1jd5VyQD7Rbw2QVBXIfw3hd1H
To: /kaggle/working/ls_test_embeddings.pkl
100%|██████████████████████████████████████| 12.4M/12.4M [00:01<00:00, 11.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Fs0PTy_xPsoX5bg4QTCrX6JAaQCHRzis
To: /kaggle/working/train.pkl
100%|█████████████████████████████████████

In [3]:
import pandas as pd
import nltk
import string
import re
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.isri import ISRIStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report
from sklearn.svm import SVC
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import torch
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from collections import OrderedDict
import matplotlib.pyplot as plt
from datasets import load_dataset
from sklearn.preprocessing import OneHotEncoder

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
label_to_class = {
    0: 'none',
    1: 'anger',
    2: 'joy',
    3: 'sadness',
    4: 'love',
    5: 'sympathy',
    6: 'surprise',
    7: 'fear'
}
classes = ['none', 'anger', 'joy', 'sadness', 'love', 'sympathy', 'surprise', 'fear']

In [5]:
with open('/kaggle/working/train.pkl', 'rb') as f:
    train = pickle.load(f)

with open('/kaggle/working/test.pkl', 'rb') as f:
    test = pickle.load(f)

with open('/kaggle/working/rs_train_embeddings.pkl', 'rb') as f:
    rs_train_embeddings = pickle.load(f)
    
with open('/kaggle/working/rs_test_embeddings.pkl', 'rb') as f:
    rs_test_embeddings = pickle.load(f)
    
with open('/kaggle/working/ls_train_embeddings.pkl', 'rb') as f:
    ls_train_embeddings = pickle.load(f)
    
with open('/kaggle/working/ls_test_embeddings.pkl', 'rb') as f:
    ls_test_embeddings = pickle.load(f)

In [6]:
X_train = ls_train_embeddings.reshape(ls_train_embeddings.shape[0], 1, ls_train_embeddings.shape[1]) # reshape input to allow for GRU
X_test = ls_test_embeddings.reshape(ls_test_embeddings.shape[0], 1, ls_test_embeddings.shape[1]) # reshape input to allow for GRU

In [7]:
encoder = OneHotEncoder()

y_train = encoder.fit_transform(train['label'].values.reshape(-1,1)).toarray()
y_test = encoder.transform(test['label'].values.reshape(-1,1)).toarray()

In [8]:
BATCH_SIZE = 128
EPOCHS = 50
EMBED_SIZE = 300
LEARNING_RATE =  0.001

early_stopping_monitor = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=10,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True)

In [11]:
def create_gru_model(model_type):
    model = Sequential()
#     model.add(Embedding(VOCAB_SIZE, EMBED_SIZE, input_length=max_length, trainable=True))
    if model_type==1:
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True)))
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(512, return_sequences=True)))
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(8, activation='softmax'))
    elif model_type==2:
        model.add(tf.keras.layers.GRU(512, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
        model.add(tf.keras.layers.GRU(256, return_sequences=True))
        model.add(tf.keras.layers.GRU(128, return_sequences=False))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(8, activation='softmax'))
    elif model_type==3:
        model.add(tf.keras.layers.GRU(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(8, activation='softmax'))
    elif model_type==4:
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Flatten())
        model.add(Dense(8, activation='softmax'))
    elif model_type==5:
        model.add(tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=False)))
        model.add(Dropout(0.5))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(8, activation='softmax'))
    
    opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    # model.summary()

    history = model.fit(X_train, np.asarray(y_train), validation_data=(X_test, np.asarray(y_test)), batch_size=BATCH_SIZE, epochs=EPOCHS, callbacks=[early_stopping_monitor])
    return model, history

In [12]:
for i in range(1,6):
    model, _ = create_gru_model(i)
    predictions = model.predict(X_test)
    print('--------------------------------------------------------')
    print()
    print('CLassification report for model {}: '.format(i))
    print(classification_report(y_test.argmax(axis=1), predictions.argmax(axis=1), target_names=classes))
    print()
    print('--------------------------------------------------------')

Epoch 1/50
63/63 [==============================] - 20s 45ms/step - loss: 1.6584 - accuracy: 0.3990 - val_loss: 1.1064 - val_accuracy: 0.6339
Epoch 2/50
63/63 [==============================] - 1s 13ms/step - loss: 1.1609 - accuracy: 0.6200 - val_loss: 0.9670 - val_accuracy: 0.6865
Epoch 3/50
63/63 [==============================] - 1s 14ms/step - loss: 0.9870 - accuracy: 0.6884 - val_loss: 0.9206 - val_accuracy: 0.7079
Epoch 4/50
63/63 [==============================] - 1s 13ms/step - loss: 0.8885 - accuracy: 0.7239 - val_loss: 0.9429 - val_accuracy: 0.7054
Epoch 5/50
63/63 [==============================] - 1s 14ms/step - loss: 0.8215 - accuracy: 0.7429 - val_loss: 0.9662 - val_accuracy: 0.7129
Epoch 6/50
63/63 [==============================] - 1s 13ms/step - loss: 0.7399 - accuracy: 0.7777 - val_loss: 1.0091 - val_accuracy: 0.7134
Epoch 7/50
63/63 [==============================] - 1s 18ms/step - loss: 0.6979 - accuracy: 0.7851 - val_loss: 1.0436 - val_accuracy: 0.7000
Epoch 8/50
6